In [312]:
import pandas as pd
df = pd.read_csv('Desktop/train.csv')
df_test = pd.read_csv('Desktop/test.csv')
df_result = pd.read_csv('Desktop/gender_submission.csv')
df.head(15)
df_train.head()
df_test.count()

PassengerId    418
Pclass         418
Name           418
Sex            418
Age            332
SibSp          418
Parch          418
Ticket         418
Fare           417
Cabin           91
Embarked       418
dtype: int64

In [313]:
print(df.isnull().sum())
df.describe()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [314]:
import category_encoders as ce

cat_columns = ['Sex','Embarked']
cat_encoder = ce.OneHotEncoder(cols=cat_columns)
cat_encoder.fit(df[cat_columns],df['Survived'])
df = df.join(cat_encoder.transform(df[cat_columns]).add_suffix('_cb'))

# cat_columns = ['Sex','Embarked']
# cat_encoder = ce.CatBoostEncoder(cols=cat_columns)
# cat_encoder.fit(df_test[cat_columns],df_test['Survived'])
df_test = df_test.join(cat_encoder.transform(df_test[cat_columns]).add_suffix('_cb'))


# from sklearn.impute import SimpleImputer

# my_imputer = SimpleImputer()
# df_impute = pd.DataFrame(my_imputer.fit_transform(df))
# df_impute.columns = df.columns
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_1_cb,Sex_2_cb,Embarked_1_cb,Embarked_2_cb,Embarked_3_cb,Embarked_4_cb
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0,1,0,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,1,0,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,1,1,0,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,0,1,0,0,0


In [315]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df[['Sex_1_cb','Sex_2_cb','Pclass','Age','SibSp','Embarked_1_cb','Embarked_2_cb','Embarked_3_cb']],df['Survived'],random_state=0)

In [318]:
#mfunction used for stacking
def model_prediction(model,train_x,train_y,test_x):
    model.fit(train_x,train_y)
    predict_train = model.predict(train_x)
    predict_test = model.predict(test_x)
    return predict_train,predict_test

#imputing missing values
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
X_impute_train = pd.DataFrame(my_imputer.fit_transform(X_train))
X_impute_test = pd.DataFrame(my_imputer.transform(X_test))
df_test = pd.DataFrame(my_imputer.transform(df_test[['Sex_1_cb','Sex_2_cb','Pclass','Age','SibSp','Embarked_1_cb','Embarked_2_cb','Embarked_3_cb']]))
X_impute_train.columns = X_train.columns
X_impute_test.columns = X_test.columns
X_n_train = X_impute_train.copy()
X_n_test = X_impute_test.copy()
X_n_test.head()

,Sex_1_cb,Sex_2_cb,Pclass,Age,SibSp,Embarked_1_cb,Embarked_2_cb,Embarked_3_cb
0,1.0,0.0,3.0,29.9,0.0,0.0,1.0,0.0
1,1.0,0.0,3.0,29.9,0.0,1.0,0.0,0.0
2,1.0,0.0,3.0,7.0,4.0,0.0,0.0,1.0
3,0.0,1.0,1.0,29.9,1.0,0.0,1.0,0.0
4,0.0,1.0,3.0,29.0,0.0,0.0,1.0,0.0


In [319]:
#creating an ensemble using three different classifier(stacking)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scl =  StandardScaler()
X_n_train = scl.fit_transform(X_n_train)
X_n_test = scl.transform(X_n_test)
lr = SVC(kernel='rbf',gamma=0.1,C=0.01)
lr_train,lr_test = model_prediction(lr,X_n_train,y_train,df_test)

dtree = DecisionTreeClassifier(max_depth=2,max_features=5)
tree_train,tree_test = model_prediction(dtree,X_impute_train,y_train,df_test)

forest = RandomForestClassifier(n_estimators=10,max_features=5,max_depth=5)
forest_train,forest_test = model_prediction(forest,X_impute_train,y_train,df_test)

train_prediction = {'lr':lr_train,'dtree':tree_train,'forest':forest_train}
train_pred = pd.DataFrame(train_prediction)
test_prediction = {'lr':lr_test,'dtree':tree_test,'forest':forest_test}
test_pred = pd.DataFrame(test_prediction)

model = LogisticRegression()
model.fit(train_pred,y_train)
result = model.predict(test_pred)
# roc_auc_score(y_test,forest_test)
# 
# model.score(test_pred,y_test)
result = pd.DataFrame(result)
result


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,0
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,0
8,1
9,0


In [320]:
result.count()

0    418
dtype: int64

In [321]:
df_Result = df_result.join(result)
df_Result.columns

Index(['PassengerId', 'Survived', 0], dtype='object')

In [322]:
df_Result.drop(columns=['Survived'],inplace=True)
df_Result.set_index('PassengerId',inplace=True)
df_Result.head()
df_Result.rename(columns={0: 'Survived'},inplace=True)

In [323]:
df_Result.to_csv('Desktop/gender_submission.csv')